# Naive Bayes Models

In this lab you will work with **naive Bayes models**. Naive Bayes models are a surprisingly useful and effective simplification of the general Bayesian models. Naive Bayes models make the naive assumption of statistical independence of the features. In many cases, naive Bayes module are surprisingly effective despite violating the assumption of independence. 

In simple terms, naive Bayes models use empirical distributions of the features to compute probabilities of the labels. The naive Bayes models can use most any family of distributions for the features. It is important to select the correct distribution family for the data you are working with. Common cases are:
- **Gaussian;** for continuous or numerical features.
- **Bernoulli;** for features with binary values. 
- **Multinomial;** for features with more than two categories. 

These is one pit fall, the model fails if a zero probability is encountered. This situation occurs when there is a 'hole' in the sample space where there are no samples. A simple smoothing procedure can deal with this problem. The smoothing hyperparameter, usually called alpha, is one of the few required for naive Bayes models. 

Some properties of naive Bayes models are:
- Computational complexity is linear in number of parameter/features, making naive Bayes models highly scalable. There are out or core approaches suitable for massive datasets.
- Requires minimal data to produce models that generalizes well. If there are only a few cases per category to train a model a naive Bayes model can be a good choice. 
- Have a simple and inherent regularization.

Naive Bayes models are used in many situations including:

- Document classification
- SPAM detection
- Image classification 



## Example: German Credit Dataset

You will try a more complex example using the credit scoring data. You will use the prepared data which had the following preprocessing:
1. Cleaning missing values.
2. Aggregating categories of certain categorical variables. 
3. Encoding categorical variables as binary dummy variables.
4. Standardizing numeric variables. 

As a first step, execute the code in the cell below to load the required packages to run the rest of this notebook. 

In [2]:
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB, BernoulliNB
#from statsmodels.api import datasets
from sklearn import datasets ## Get dataset from sklearn
import sklearn.model_selection as ms
import sklearn.metrics as sklm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.random as nr

%matplotlib inline

Execute the code in the cell below to load the features and labels as numpy arrays for the example. 

In [3]:
Features = np.array(pd.read_csv('Credit_Features.csv'))
Labels = np.array(pd.read_csv('Credit_Labels.csv'))
Labels = Labels.reshape(Labels.shape[0],)
print(Features.shape)
print(Labels.shape)

(1000, 35)
(1000,)


The Features array has both numeric features and binary features (dummy variables for the categorical features). Therefore, a Gaussian model must be used. However, this model is not ideal, since numeric features are mixed with features exhibiting Bernoulli distributions, the binary features. 

The code in the cell below does the following processing:
1. Defines a 10 fold cross validation object. 
2. Defines a Gaussian naive Bayes model.
3. Performs a 10 fold cross validation.
4. Prints results from the cross validation. 

Execute this code and examine the result. 

In [4]:
print(Features[:3,])

[[-1.86486906 -0.933901    0.91847717  2.27100592  0.          1.
   0.          0.          0.          1.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          1.          0.          0.          0.
   0.          0.          1.          0.          0.          0.
   1.          0.          0.          1.          0.        ]
 [ 1.7083687   1.16304581 -0.87018333 -1.44615223  1.          0.
   0.          0.          0.          0.          1.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          1.          0.          0.          1.
   0.          0.          0.          0.          1.          0.
   0.          0.          0.          1.          0.        ]
 [-0.67378981 -0.1815588  -0.87018333  1.22669602  0.          0.
   0.          1.          0.          1.          0.          0.
   0.          0.          0.          0.          0.          1.
   0.          0

In [5]:
nr.seed(321)
cv_folds = ms.KFold(n_splits=10, shuffle = True)
    
nr.seed(498)
NB_credit = GaussianNB()
cv_estimate = ms.cross_val_score(NB_credit, Features, Labels, 
                                 cv = cv_folds) # Use the outside folds

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

Mean performance metric = 0.691
SDT of the metric       = 0.093
Outcomes by cv fold
Fold  1    0.720
Fold  2    0.660
Fold  3    0.770
Fold  4    0.690
Fold  5    0.750
Fold  6    0.430
Fold  7    0.690
Fold  8    0.760
Fold  9    0.710
Fold 10    0.730


In [6]:
nr.seed(498)
NB_credit = GaussianNB()
cv_estimate = ms.cross_val_score(NB_credit, Features, Labels, 
                                 cv = 10) # Use the outside folds

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

Mean performance metric = 0.679
SDT of the metric       = 0.106
Outcomes by cv fold
Fold  1    0.670
Fold  2    0.390
Fold  3    0.680
Fold  4    0.710
Fold  5    0.760
Fold  6    0.770
Fold  7    0.620
Fold  8    0.760
Fold  9    0.730
Fold 10    0.700


Examine these results. Notice that the standard deviation of the mean of the AUC is more than an order of magnitude smaller than the mean. This indicates that this model is likely to generalize well. 

Now, you will build and test a model using a single split of the dataset. As a first step, execute the code in the cell below to create training and testing dataset.

In [7]:
## Randomly sample cases to create independent training and test data
nr.seed(1115)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 300)
X_train = Features[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
X_test = Features[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

The code in the cell below defines a naive Bayes model object and then fits the model to the training data. Execute this code:

In [8]:
NB_credit_mod = GaussianNB() 
NB_credit_mod.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

Using the test data subset, the code in the cell below scores and prints evaluation metrics for the model. 

Execute this code, examine the results, and answer **Question 2** on the course page. 

In [9]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])

def print_metrics(labels, probs, threshold):
    scores = score_model(probs, threshold)
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    print('AUC             %0.2f' % sklm.roc_auc_score(labels, probs[:,1]))
    print('Macro precision %0.2f' % float((float(metrics[0][0]) + float(metrics[0][1]))/2.0))
    print('Macro recall    %0.2f' % float((float(metrics[1][0]) + float(metrics[1][1]))/2.0))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])
    
probabilities = NB_credit_mod.predict_proba(X_test)
print_metrics(y_test, probabilities, 0.5)    

                 Confusion matrix
                 Score positive    Score negative
Actual positive        30               182
Actual negative         6                82

Accuracy        0.37
AUC             0.69
Macro precision 0.57
Macro recall    0.54
 
           Positive      Negative
Num case      212            88
Precision    0.83          0.31
Recall       0.14          0.93
F1           0.24          0.47


Overall, these performance metrics are poor. Barely half the bad credit risk customers are correctly identified. The reported AUC is quite a bit better than the mean achieved with the 5 fold cross validation. It is likely these figures are optimistic. 

Perhaps, a Bernoulli naive Bayes model will work better. Since naive Bayes models tend to be less sensitive to the quantity of training data, this approach may be reasonable. To apply this model, the numeric features must be dropped from the array. Execute the code in the cell below to remove the numeric features and examine the resulting sample. 

In [10]:
Features = Features[:,4:]
Features[:3,:]

array([[0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.]])

In [11]:
print(Features[:3,:])

[[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 1. 0. 0. 0. 1. 0.]]


You can see that all of the features are binary. A Bernoulli naive Bayes can now be applied to these features. 

10 fold nested cross validation is used to estimate the optimal hyperparameter and performs model selection for the naive Bayes model. Execute the code in the cell below to define inside and outside fold objects.

In [12]:
nr.seed(123)
inside = ms.KFold(n_splits=10, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=10, shuffle = True)

The code in the cell below estimates the best hyperparameters using 10 fold cross validation. There are two points to notice here:
1. In this case, a grid for one hyperparameter: alpha is the smoothing parameter to avoid zero probabilities.  
3. The model is fit on the grid and the best estimated hyperparameters are printed. 

Execute this code and examine the result. 

In [13]:
nr.seed(3456)
## Define the dictionary for the grid search and the model object to search on
param_grid = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]}
## Define the NB regression model
NB_clf = BernoulliNB() 

## Perform the grid search over the parameters
clf = ms.GridSearchCV(estimator = NB_clf, param_grid = param_grid, 
                      cv = inside, # Use the inside folds
                      scoring = 'roc_auc',
                      return_train_score = True)
clf.fit(Features, Labels)
print(clf.best_estimator_.alpha)

1


The smallest alpha hyperparameter has been selected. This indicates that there is very little problem with zero probabilities in this problem. This situation results from the fact that the probability space sampled is dense. 

The code in the cell below executes the outer loop of the cross validation to estimate model performance with the optimal hyperparameter. Execute this code and examine the result.

In [14]:
#NB_credit = BernoulliNB(alpha = clf.best_estimator_.alpha)
nr.seed(498)
cv_estimate = ms.cross_val_score(clf, Features, Labels, 
                                 cv = outside) # Use the outside folds

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

Mean performance metric = 0.754
SDT of the metric       = 0.040
Outcomes by cv fold
Fold  1    0.735
Fold  2    0.701
Fold  3    0.733
Fold  4    0.745
Fold  5    0.771
Fold  6    0.757
Fold  7    0.762
Fold  8    0.857
Fold  9    0.765
Fold 10    0.713


At first look, the AUC seems reasonable. The standard deviation is an order of magnitude less than the AUC. 

Next, sample the dataset into training and testing subsets by executing the code in the cell below.

In [15]:
## Randomly sample cases to create independent training and test data
nr.seed(1115)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 300)
X_train = Features[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
X_test = Features[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

Now, execute the code below to fit and score the Bernoulli naive Bayes model and display the performance metrics. 

In [16]:
NB_credit_mod = BernoulliNB(alpha = clf.best_estimator_.alpha) 
NB_credit_mod.fit(X_train, y_train)
probabilities = NB_credit_mod.predict_proba(X_test)
print_metrics(y_test, probabilities, 0.5)    

                 Confusion matrix
                 Score positive    Score negative
Actual positive       178                34
Actual negative        34                54

Accuracy        0.77
AUC             0.78
Macro precision 0.73
Macro recall    0.73
 
           Positive      Negative
Num case      212            88
Precision    0.84          0.61
Recall       0.84          0.61
F1           0.84          0.61


The results for this Bernoulli naive Bayes model are much better than for the Gaussian model. Still, they could be better.

The current model uses the empirical distribution of the label values for the prior value of $p$ of the Bernoulli distribution. This probability is invariably skewed toward the majority case. Since the bank cares more about the minority case, setting this distribution to a fixed prior value can help overcome the class imbalance. The code in the cell below redefines the model object with prior probability of 0.6 for the minority case. 

Execute this code, examine the results, and answer **Question 3** on the course page.

In [17]:
NB_credit_mod = BernoulliNB(alpha = clf.best_estimator_.alpha,
                            class_prior = [0.4,0.6]) 
NB_credit_mod.fit(X_train, y_train)
probabilities = NB_credit_mod.predict_proba(X_test)
print_metrics(y_test, probabilities, 0.5)    

                 Confusion matrix
                 Score positive    Score negative
Actual positive       116                96
Actual negative        17                71

Accuracy        0.62
AUC             0.78
Macro precision 0.65
Macro recall    0.68
 
           Positive      Negative
Num case      212            88
Precision    0.87          0.43
Recall       0.55          0.81
F1           0.67          0.56


The majority of bad credit cases are now correctly identified. However, this characteristic is at the cost of a high false negative error rate. Still, given that the cost to the bank of a false negative is five times the cost of a false positive, this may be a good solution. An infinite number of other models are possible by changing the prior distribution. 

## Summary

In this lab you have accomplished the following:
1. Used a Gaussian naive model to classify the cases of the iris data. The overall model performance was reasonable. 
2. Fit a Gaussian naive Bayes model on the bank credit data. The performance of this model was poor as a result of many Bernoulli distributed dummy features.
3. Used a Bernoulli naive Bayes model for the bank credit data by eliminating the numeric features. Overall, this model was much better. 
4. A model skewed toward detecting bad credit cases was created using a prior distribution rather than the empirical distribution of the labels. This model correctly classified a significant number of the bad credit cases. Selecting other prior distributions will give other models. 